# My Function Zoo

This notebook curates a list of helper functions. When a project finishes, reusable functions are moved here.

In [ ]:
# Numpy
import numpy as np
import pandas as pd
%matplotlib nbagg
import matplotlib.pyplot as plt
from matplotlib import gridspec
import scipy.stats

# Machine learning
import tensorflow as tf
import keras
import sklearn
import sklearn.preprocessing, sklearn.base, sklearn.utils, sklearn.model_selection

# Various Python tricks and libraries
import requests
import time
import functools
import operator
import collections
from tqdm import tqdm, tqdm_notebook, tnrange
import dill as pickle
import IPython

# Parallel
import joblib

## Python

### Iterable to list (low performance, please avoid!)

In [ ]:
def is_iterable(L):
    return hasattr(L, '__iter__')

def to_iterable(L):
    if isinstance(L, (np.ndarray, np.generic)):
        return L.tolist()
    if isinstance(L, pd.DataFrame):
        return L.values.tolist()
    if is_iterable(L):
        return recursive_map(lambda x:x, L)
    raise ValueError
    
def flatten(L):
    return reduce(operator.add, map(lambda l: flatten(l) if is_iterable(l) else [l], L))

def recursive_map(func, L):
    return map(lambda l: recursive_map(func, l) if is_iterable(l) else func(l), L)

def get_index(item, L, index_unexpected=-1, random_unexpected=0.): # first occurence or -1
    return index_unexpected if np.random.rand() < random_unexpected or item not in L else np.argmax(np.array(L)==item)

def get_value(index, L, value_unexpected_index=None, index_unexpected=-1): # L[index] or None
    return value_unexpected_index if index == index_unexpected else L[index]


## Machine Learning

### Categorical data (low performance, please avoid!)

In [ ]:
class LabelEncoder(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin): 
    # sklearn.preprocessing.LabelEncoder with irregular 2D array, unexpected index or value, and random -1 return.
    
    def __init__(self, index_unexpected=-1, random_unexpected=0., value_unexpected_index=None):
        self.index_unexpected = index_unexpected
        self.random_unexpected = random_unexpected
        self.value_unexpected_index = value_unexpected_index
        
    def fit(self, y):
        y_flattened = flatten(y)
        self.classes_ = np.unique(y_flattened)
        return self

    def transform(self, y):
        func = functools.partial(get_index, 
                                 L=self.classes_, 
                                 index_unexpected=self.index_unexpected, 
                                 random_unexpected=self.random_unexpected)
        return recursive_map(
            func=lambda item: get_index(
                item=item, 
                L=self.classes_, 
                index_unexpected=self.index_unexpected, 
                random_unexpected=self.random_unexpected),
            L=y)

    def inverse_transform(self, y):
        return recursive_map(
            func=lambda index: get_value(
                index=index, 
                L=self.classes_, 
                index_unexpected=self.index_unexpected, 
                random_unexpected=self.random_unexpected),
            L=y)
    

### Regression

In [ ]:
def r2_score(ytrue, ypred): # sklearn.metrics.r2_score in tensorflow. 1 output only. 

    ytrue_mean = tf.reduce_mean(ytrue, name="ytrue_mean")
    r2_score = tf.subtract(1., tf.truediv(tf.reduce_mean((ytrue - ypred) ** 2), tf.reduce_mean((ytrue - ytrue_mean) ** 2)), name="r2_score")
    return r2_score

class TqdmProgBar(keras.callbacks.Callback):
    '''features:
    1. tqdm ETA bar
    2. logs[field] plotted for each field in fields
    '''
    
    def __init__(self, n_epochs, fields): 
        self.n_epochs = n_epochs
        self.fields = fields
        
        self.fields_history = dict((field, []) for field in fields)
        self.fig, self.ax = plt.subplots(1, 1)
        
    def on_train_begin(self, logs):
        self.pbar = tqdm_notebook(total=self.n_epochs, leave=False)
        
    def on_train_end(self, logs):
        self.pbar.close()
        
    def on_epoch_end(self, epoch, logs, log_interval = 40):
        if epoch % log_interval == 0:
            self.pbar.update(log_interval)

            for field in self.fields:
                self.fields_history[field].append(logs[field])

            self.ax.clear()
            for field in self.fields:
                self.ax.plot(self.fields_history[field], label="%s=%.2f" %(field, self.fields_history[field][-1]))
            self.ax.legend(loc='best')
            self.fig.canvas.draw()
        

### sklearn

In [ ]:
class Pipeline():
    '''
    Same as sklearn's pipeline except:
    - does not require 2 inputs and outputs
    - requires only fit() and transform()
    - returns state list
    - not too inefficient
    '''
    
    def __init__(self, ETs):
        self.ETs = Estimators_Transformers = ETs
      
    def fit(self, *args):
        '''Note: does not take keyworded input.'''
        for ET in self.ETs:
            args = ET.fit(*args).transform(*args)
        return self

    def transform(self, *args):
        for ET in self.ETs:
            args = ET.transform(*args)
        return args
    
    def transforms(self, *args):
        ET_s = []
        for ET in self.ETs:
            args = ET.transform(*args)
            ET_s.append(args)
        return ET_s
    
    def fit_transform(self, *args):
        for ET in self.ETs:
            args = ET.fit(*args).transform(*args)
        return args
    
    def fit_transforms(self, *args):
        ET_s = []
        for ET in self.ETs:
            args = ET.fit(*args).transform(*args)
            ET_s.append(args)
        return ET_s